# section8

## 機械学習の適用１　－感情分析

### POINT
- テキストデータの**クレンジングと準備**
- テキスト文書から**特徴ベクトルの構築**
- 映画レビューを肯定的な文章と否定的な文章に分類する機械学習モデルの構築
- **アウトオブコア**による大規模なテキストデータセットの処理
- 文章コレクションからカテゴリのトピックを推定する

感情分析とは？  
→自然言語処理（NLP）の一分野。機械学習アルゴリズムを使用し、極性（polarity）に基づいて文書を分類する。

In [6]:
import tarfile
with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as tar:
    tar.extractall()

In [13]:
#import puprind
import pandas as pd
import os

In [14]:
basepath = 'aclImdb'
labels = {'pos':1, 'neg':0}
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in os.listdir(path):
            with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)

df.columns = ['review', 'sentiment']

KeyboardInterrupt: 

In [ ]:
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

In [15]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


## BoWモデルとは？
1. 文書の全体集合から、例えば単語という一意なトークンからなる語彙を作成する
2. 各文書での各単語の出現回数を含んだ特徴ベクトルを構築する。

In [18]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining, the weather is sweet, and one and one is two'
])
bag = count.fit_transform(docs)

In [19]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [20]:
bag.toarray()

array([[0, 1, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 1],
       [2, 3, 2, 1, 1, 1, 2, 1, 1]], dtype=int64)

In [21]:
print(count.get_feature_names())

['and', 'is', 'one', 'shining', 'sun', 'sweet', 'the', 'two', 'weather']


連続するアイテムはnグラムと呼ばれる。サイズが３，４のｎグラムはメールのアンチスパムフィルタリングで優れた性能を発揮する。CounterVectorizerクラスではngram_range=(2,2)で初期化することにより、２グラム表現に切り替えることができる。

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


↑l2正則化しているので単語の多い文章では'the'のようにほかの文書よりも出現回数が多いにも関わらず値が小さくなることもある。

### テキストデータのクレンジング

In [23]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [24]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ''.join(emoticons).replace('-', ''))
    return text

In [25]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [26]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :):(:)'

In [27]:
df['review'] = df['review'].apply(preprocessor)

In [28]:
def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [29]:
from nltk.stem.porterrter import PorterStemmer

ModuleNotFoundError: No module named 'nltk.stem.porterrter'

In [30]:
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tokenizer_porter('runner like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

Poterアルゴリズムは最も古く最も単純なステミングアルゴリズムである。最近ではSnowballステマーやLancasterステマーがある。見出し語化と呼ばれる手法は文法的に正しく取得することを目指す。ただし、見出し語化はステミングより計算の負荷が高い。また、テキスト分類の性能にはさほど影響を与えないという実験結果も得られている。

#### ストップワードの除去
ごくありふれた単語のこと。is,and,has,likeなど。これが役に立つのは、TF-IDEではなく、生の単語の出現頻度か正規化された単語の出現頻度を扱っている場合である。

In [31]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hiroyuki\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

↑ストップワードの取得

In [32]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

↑英語のストップワードを取得し、適用。

In [33]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [36]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,# 小文字への変換(default=True)
                        preprocessor=None)
param_grid = [{'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf': [False],
               'vect__norm': [None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]}]
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5, verbose=1, #途中経過を示す
                           n_jobs=1)
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


KeyboardInterrupt: 

分析の流れ
1. テキストデータのクレンジング  
正規表現などを使って句読点、HTMLマークアップ、絵文字などを削除ないしは変換した。


2. 文書のトークン化（個々の単語に分けること）  
便利な方法にワードステミングがある。running→runのように変換できる。PorterStemmerアルゴリズムなどを適用する。ストップワードの除去も場合によっては行う。


3. 単語を特徴ベクトルに変換する  
 tf(t,d)←tは単語の出現回数、dは文書dにおける。  
(raw **t**erm **f**requencies 生の出現頻度)  
 tf-idf(t,d)  
(**t**erm **f**requency **i**nverse **d**ocument **f**requency)出現頻度によって重みづけできる。  
idf = log(n/(1+df))←nは文書の総数、dfは単語tを含んでいる文書dの個数。


In [39]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-','')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [64]:
def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv)
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [65]:
next(stream_docs(path='movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [66]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [79]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
clf = SGDClassifier(loss='log', random_state=1, n_iter=1)
doc_stream = stream_docs(path='movie_data.csv')

In [80]:
classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    

In [81]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f' % clf.score(X_test, y_test))

Accuracy: 0.867


### 潜在ディリクレ配分（LDA）によるトピックモデルの構築
ベイズ推定に関する知識が要求される。LDAは生成的確率モデル。トピックの個数を事前に定義しなければならない。

In [83]:
import pandas as pd

In [84]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')

In [85]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer(stop_words='english',
                        max_df=.1,
                        max_features=5000)
X = count.fit_transform(df['review'].values)

In [86]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_topics=10,
                                random_state=123,
                                learning_method='batch')
X_topics = lda.fit_transform(X)